<a id='head'></a>
# Шаблоны работы с данными

```puthon
sql.SQL(',').join([sql.Placeholder()] * len(headers))
```

# Оглавление

- [**1. Заливка данных**](#part_01)
    - [1.1 Заливка CSV чанками с заголовками полей](#part_11)
    - [1.2 Заливка CSV чанками без заголовков полей](#part_12)
    - [1.3 Решение инкрементной загрузки CSV через onETL с заголовками в исходном файле](#part_13)
    - [1.4 Решение инкрементной загрузки CSV через onETL без заголовков в исходном файле](#part_14)
    - [1.5 ](#part_15)
    - [1.6 ](#part_16)
    - [1.7 ](#part_17)
    - [1.8 ](#part_18)
    - [1.9 ](#part_19)
    
- [**2.**](#part_02)
    - [2.1 ](#part_21)
    - [2.2 ](#part_22)
    - [2.3 ](#part_23)
    - [2.4 ](#part_24)
    - [2.5 ](#part_25)
    - [2.6 ](#part_26)
    - [2.7 ](#part_27)
    - [2.8 ](#part_28)
    - [2.9 ](#part_29)
    
- [**3.**](#part_03)
    - [3.1 ](#part_31)
    - [3.2 ](#part_32)
    - [3.3 ](#part_33)
    - [3.4 ](#part_34)
    - [3.5 ](#part_35)
    - [3.6 ](#part_36)
    - [3.7 ](#part_37)
    - [3.8 ](#part_38)
    - [3.9 ](#part_39)

- [**4.**](#part_04)
    - [4.1 ](#part_41)
    - [4.2 ](#part_42)
    - [4.3 ](#part_43)
    - [4.4 ](#part_44)
    - [4.5 ](#part_45)
    - [4.6 ](#part_46)
    - [4.7 ](#part_47)
    - [4.8 ](#part_48)
    - [4.9 ](#part_49)

- [**5.**](#part_05)
    - [5.1 ](#part_51)
    - [5.2 ](#part_52)
    - [5.3 ](#part_53)
    - [5.4 ](#part_54)
    - [5.5 ](#part_55)
    - [5.6 ](#part_56)
    - [5.7 ](#part_57)
    - [5.8 ](#part_58)
    - [5.9 ](#part_59)
    
- [**6.**](#part_06)
    - [6.1 ](#part_61)
    - [6.2 ](#part_62)
    - [6.3 ](#part_63)
    - [6.4 ](#part_64)
    - [6.5 ](#part_65)
    - [6.6 ](#part_66)
    - [6.7 ](#part_67)
    - [6.8 ](#part_68)
    - [6.9 ](#part_69)

- [**7.**](#part_07)
    - [7.1 ](#part_71)
    - [7.2 ](#part_72)
    - [7.3 ](#part_73)
    - [7.4 ](#part_74)
    - [7.5 ](#part_75)
    - [7.6 ](#part_76)
    - [7.7 ](#part_77)
    - [7.8 ](#part_78)
    - [7.9 ](#part_79)

- [**8.**](#part_08)
    - [8.1 ](#part_81)
    - [8.2 ](#part_82)
    - [8.3 ](#part_83)
    - [8.4 ](#part_84)
    - [8.5 ](#part_85)
    - [8.6 ](#part_86)
    - [8.7 ](#part_87)
    - [8.8 ](#part_88)
    - [8.9 ](#part_89)

- [**9.**](#part_09)
    - [9.1 ](#part_91)
    - [9.2 ](#part_92)
    - [9.3 ](#part_93)
    - [9.4 ](#part_94)
    - [9.5 ](#part_95)
    - [9.6 ](#part_96)
    - [9.7 ](#part_97)
    - [9.8 ](#part_98)
    - [9.9 ](#part_99)

---

<a id='part_01'></a>
# 1. Заливка данных [▴](#head)

<a id='part_11'></a>
## 1.1 Заливка CSV чанками с заголовками полей [▴](#head)

```python
import pandas as pd
import psycopg2
from psycopg2 import sql

# Параметры подключения к базе данных
conn_params = {
    'dbname': 'your_database',
    'user': 'your_username',
    'password': 'your_password',
    'host': 'your_host',
    'port': 'your_port'
}

# Имя таблицы в PostgreSQL
table_name = 'your_table'

# Путь к CSV файлу
csv_file_path = 'path/to/your/large_file.csv'

# Размер чанка (количество строк для обработки за один раз)
chunksize = 10000

# Подключение к базе данных
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

# Чтение CSV файла по частям
for chunk in pd.read_csv(csv_file_path, chunksize=chunksize):
    # Преобразование DataFrame в список кортежей
    tuples = [tuple(x) for x in chunk.to_numpy()]
    # Получение списка колонок
    cols = ','.join(list(chunk.columns))
    # SQL запрос для вставки данных
    query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
        sql.Identifier(table_name), # Возвращает имя таблицы
        sql.SQL(cols),              # Возвращает имена столбцов
        sql.SQL(',').join(map(sql.Placeholder, chunk.columns)) # Возвращает нужное количество плейсхолдеров
    )
    # Выполнение запроса
    cursor.executemany(query, tuples)
    # Фиксация изменений
    conn.commit()

# Закрытие соединения
cursor.close()
conn.close()

```

<a id='part_12'></a>
## 1.2 Заливка CSV чанками без заголовков полей [▴](#head)

```python

import pandas as pd
import psycopg2
from psycopg2 import sql

# Параметры подключения к базе данных
conn_params = {
    'dbname': 'your_database',
    'user': 'your_username',
    'password': 'your_password',
    'host': 'your_host',
    'port': 'your_port'
}

# Имя таблицы в PostgreSQL
table_name = 'your_table'

# Путь к CSV файлу
csv_file_path = 'path/to/your/large_file.csv'

# Список заголовков
headers = ['id', 'user', 'name', 'phone']

# Размер чанка (количество строк для обработки за один раз)
chunksize = 10000

# Подключение к базе данных
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

# Чтение CSV файла по частям
for chunk in pd.read_csv(csv_file_path, chunksize=chunksize, header=None, names=headers):
    # Преобразование DataFrame в список кортежей
    tuples = [tuple(x) for x in chunk.to_numpy()]
    
    # Получение списка колонок
    cols = ','.join(headers)
    
    # SQL запрос для вставки данных
    query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
        sql.Identifier(table_name),
        sql.SQL(cols),
        sql.SQL(',').join(map(sql.Placeholder, headers))
    )
    
    # Выполнение запроса
    cursor.executemany(query, tuples)
    
    # Фиксация изменений
    conn.commit()

# Закрытие соединения
cursor.close()
conn.close()
```


<a id='part_13'></a>
## 1.3 Решение инкрементной загрузки CSV через onETL с заголовками в исходном файле  [▴](#head)

```python
from pyspark.sql import SparkSession
from onetl.connection import WebDAV, Greenplum
from onetl.core import FileFormat
from onetl.strategy import IncrementalStrategy

# Настройки WebDAV
webdav_host = "https://example.com"
webdav_user = "your_username"
webdav_password = "your_password"
webdav_file_path = "/path/to/file.csv"  # Путь к файлу на WebDAV

# Настройки Greenplum
gp_host = "greenplum_host"
gp_port = 5432
gp_database = "your_database"
gp_user = "your_username"
gp_password = "your_password"
gp_schema = "your_schema"
gp_table = "your_table"

# Временный файл для скачивания
temp_file_path = "/path/to/temp/file.csv"

# Инициализация Spark-сессии
spark = SparkSession.builder \
    .appName("CSV to Greenplum") \
    .getOrCreate()

# Подключение к WebDAV
webdav = WebDAV(
    host=webdav_host,
    user=webdav_user,
    password=webdav_password,
)

# Скачивание файла с WebDAV
webdav.download_file(remote_path=webdav_file_path, local_path=temp_file_path)

# Чтение CSV-файла с помощью Spark
df = spark.read \
    .format("csv") \
    .option("header", "true") \  # Если в CSV есть заголовоки
    .option("inferSchema", "true") \  # Автоматическое определение типов данных
    .load(temp_file_path)

# Подключение к Greenplum через onETL
gp = Greenplum(
    host=gp_host,
    port=gp_port,
    user=gp_user,
    password=gp_password,
    database=gp_database,
)

# Загрузка данных из Spark DataFrame в Greenplum
with IncrementalStrategy(gp) as strategy:
    strategy.write_dataframe(
        df=df,
        schema=gp_schema,
        table=gp_table,
        format=FileFormat.CSV,  # Указываем формат данных
    )

# Удаляем временный файл
import os
os.remove(temp_file_path)

# Останавливаем Spark-сессию
spark.stop()

```

<a id='part_14'></a>
## 1.4 Решение инкрементной загрузки CSV через onETL без заголовков в исходном файле [▴](#head)

```python
from pyspark.sql import SparkSession
from onetl.connection import WebDAV, Greenplum
from onetl.core import FileFormat
from onetl.strategy import IncrementalStrategy

# Настройки WebDAV
webdav_host = "https://example.com"
webdav_user = "your_username"
webdav_password = "your_password"
webdav_file_path = "/path/to/file.csv"  # Путь к файлу на WebDAV

# Настройки Greenplum
gp_host = "greenplum_host"
gp_port = 5432
gp_database = "your_database"
gp_user = "your_username"
gp_password = "your_password"
gp_schema = "your_schema"
gp_table = "your_table"

# Временный файл для скачивания
temp_file_path = "/path/to/temp/file.csv"

# Инициализация Spark-сессии
spark = SparkSession.builder \
    .appName("CSV to Greenplum") \
    .getOrCreate()

# Подключение к WebDAV
webdav = WebDAV(
    host=webdav_host,
    user=webdav_user,
    password=webdav_password,
)

# Скачивание файла с WebDAV
webdav.download_file(remote_path=webdav_file_path, local_path=temp_file_path)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Определяем схему вручную
schema = StructType([
    StructField("name", StringType(), nullable=True),
    StructField("age", IntegerType(), nullable=True),
    StructField("city", StringType(), nullable=True),
])

df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \  # Используем ручную схему
    .load(temp_file_path)

# Подключение к Greenplum через onETL
gp = Greenplum(
    host=gp_host,
    port=gp_port,
    user=gp_user,
    password=gp_password,
    database=gp_database,
)

# Загрузка данных из Spark DataFrame в Greenplum
with IncrementalStrategy(gp) as strategy:
    strategy.write_dataframe(
        df=df,
        schema=gp_schema,
        table=gp_table,
        format=FileFormat.CSV,  # Указываем формат данных
    )

# Удаляем временный файл
import os
os.remove(temp_file_path)

# Останавливаем Spark-сессию
spark.stop()

```

<a id='part_15'></a>
## 1.5 [▴](#head)

<a id='part_16'></a>
## 1.6 [▴](#head)

<a id='part_17'></a>
## 1.7 [▴](#head)

<a id='part_18'></a>
## 1.8 [▴](#head)

<a id='part_19'></a>
## 1.9 [▴](#head)

---

<a id='part_02'></a>
# 2. [▴](#head)

<a id='part_21'></a>
## 2.1 [▴](#head)

<a id='part_22'></a>
## 2.2 [▴](#head)

<a id='part_23'></a>
## 2.3 [▴](#head)

<a id='part_24'></a>
## 2.4 [▴](#head)

<a id='part_25'></a>
## 2.5 [▴](#head)

<a id='part_26'></a>
## 2.6 [▴](#head)

<a id='part_27'></a>
## 2.7 [▴](#head)

<a id='part_28'></a>
## 2.8 [▴](#head)

<a id='part_29'></a>
## 2.9 [▴](#head)

---

<a id='part_03'></a>
# 3. [▴](#head)

<a id='part_31'></a>
## 3.1 [▴](#head)

<a id='part_32'></a>
## 3.2 [▴](#head)

<a id='part_33'></a>
## 3.3 [▴](#head)

<a id='part_34'></a>
## 3.4 [▴](#head)

<a id='part_35'></a>
## 3.5 [▴](#head)

<a id='part_36'></a>
## 3.6 [▴](#head)

<a id='part_37'></a>
## 3.7 [▴](#head)

<a id='part_38'></a>
## 3.8 [▴](#head)

<a id='part_39'></a>
## 3.9 [▴](#head)

---

<a id='part_04'></a>
# 4. [▴](#head)

<a id='part_41'></a>
## 4.1 [▴](#head)

<a id='part_42'></a>
## 4.2 [▴](#head)

<a id='part_43'></a>
## 4.3 [▴](#head)

<a id='part_44'></a>
## 4.4 [▴](#head)

<a id='part_45'></a>
## 4.5 [▴](#head)

<a id='part_46'></a>
## 4.6 [▴](#head)

<a id='part_47'></a>
## 4.7 [▴](#head)

<a id='part_48'></a>
## 4.8 [▴](#head)

<a id='part_49'></a>
## 4.9 [▴](#head)

---

<a id='part_05'></a>
# 5. [▴](#head)

<a id='part_51'></a>
## 5.1 [▴](#head)

<a id='part_52'></a>
## 5.2 [▴](#head)

<a id='part_53'></a>
## 5.3 [▴](#head)

<a id='part_54'></a>
## 5.4 [▴](#head)

<a id='part_55'></a>
## 5.5 [▴](#head)

<a id='part_56'></a>
## 5.6 [▴](#head)

<a id='part_57'></a>
## 5.7 [▴](#head)

<a id='part_58'></a>
## 5.8 [▴](#head)

<a id='part_59'></a>
## 5.9 [▴](#head)

---

<a id='part_06'></a>
# 6. [▴](#head)

<a id='part_61'></a>
## 6.1 [▴](#head)

<a id='part_62'></a>
## 6.2 [▴](#head)

<a id='part_63'></a>
## 6.3 [▴](#head)

<a id='part_64'></a>
## 6.4 [▴](#head)

<a id='part_65'></a>
## 6.5 [▴](#head)

<a id='part_66'></a>
## 6.6 [▴](#head)

<a id='part_67'></a>
## 6.7 [▴](#head)

<a id='part_68'></a>
## 6.8 [▴](#head)

<a id='part_69'></a>
## 6.9 [▴](#head)

---

<a id='part_07'></a>
# 7. [▴](#head)

<a id='part_71'></a>
## 7.1 [▴](#head)

<a id='part_72'></a>
## 7.2 [▴](#head)

<a id='part_73'></a>
## 7.3 [▴](#head)

<a id='part_74'></a>
## 7.4 [▴](#head)

<a id='part_75'></a>
## 7.5 [▴](#head)

<a id='part_76'></a>
## 7.6 [▴](#head)

<a id='part_77'></a>
## 7.7 [▴](#head)

<a id='part_78'></a>
## 7.8 [▴](#head)

<a id='part_79'></a>
## 7.9 [▴](#head)

---

<a id='part_08'></a>
# 8. [▴](#head)

<a id='part_81'></a>
## 8.1 [▴](#head)

<a id='part_82'></a>
## 8.2 [▴](#head)

<a id='part_83'></a>
## 8.3 [▴](#head)

<a id='part_84'></a>
## 8.4 [▴](#head)

<a id='part_85'></a>
## 8.5 [▴](#head)

<a id='part_86'></a>
## 8.6 [▴](#head)

<a id='part_87'></a>
## 8.7 [▴](#head)

<a id='part_88'></a>
## 8.8 [▴](#head)

<a id='part_89'></a>
## 8.9 [▴](#head)

---

<a id='part_09'></a>
# 9. [▴](#head)

<a id='part_91'></a>
## 9.1 [▴](#head)

<a id='part_92'></a>
## 9.2 [▴](#head)

<a id='part_93'></a>
## 9.3 [▴](#head)

<a id='part_94'></a>
## 9.4 [▴](#head)

<a id='part_95'></a>
## 9.5 [▴](#head)

<a id='part_96'></a>
## 9.6 [▴](#head)

<a id='part_97'></a>
## 9.7 [▴](#head)

<a id='part_98'></a>
## 9.8 [▴](#head)

<a id='part_99'></a>
## 9.9 [▴](#head)